In [28]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [33]:
X_train = pd.read_csv("GROUP20_DATASET/FINAL_GROUP20_TOUR_PACKAGES_TRAIN_X.csv")
X_test = pd.read_csv("GROUP20_DATASET/FINAL_GROUP20_TOUR_PACKAGES_TEST_X.csv")
y_train = pd.read_csv("GROUP20_DATASET/FINAL_GROUP20_TOUR_PACKAGES_TRAIN_Y.csv")
y_test = pd.read_csv("GROUP20_DATASET/FINAL_GROUP20_TOUR_PACKAGES_TEST_Y.csv")

In [34]:
clf = MLPClassifier(random_state=1, max_iter=500).fit(X_train.drop(['CUSTOMER_ID'], axis=1), y_train)

/Users/bharatwaajshankar/Library/Python/3.8/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [54]:
clf = RandomForestClassifier(max_depth=8, random_state=0).fit(X_train.drop(['CUSTOMER_ID'], axis=1), y_train)

<ipython-input-54-640d719dc0d4>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = RandomForestClassifier(max_depth=8, random_state=0).fit(X_train.drop(['CUSTOMER_ID'], axis=1), y_train)


In [57]:
clf.predict_proba(X_test.drop(['CUSTOMER_ID'], axis=1))

array([[5.33474262e-01, 3.57222196e-01, 8.66197183e-04, 1.05233235e-01,
        3.20410952e-03],
       [5.07551987e-01, 3.45376564e-01, 4.28678259e-04, 1.43933437e-01,
        2.70933364e-03],
       [3.60849327e-01, 4.28939493e-01, 5.13654207e-03, 1.82823824e-01,
        2.22508137e-02],
       ...,
       [2.32683928e-01, 4.82610661e-01, 4.86083649e-02, 1.36769128e-01,
        9.93279183e-02],
       [1.52470410e-01, 6.11113993e-01, 3.87365331e-02, 1.08763031e-01,
        8.89160332e-02],
       [9.16565705e-01, 5.35941700e-02, 0.00000000e+00, 2.96215454e-02,
        2.18579235e-04]])

In [59]:
y_pred = clf.predict(X_test.drop(['CUSTOMER_ID'], axis=1))

In [60]:
from sklearn.metrics import classification_report

In [61]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Basic       0.59      0.73      0.65       359
      Deluxe       0.52      0.61      0.56       363
        King       0.38      0.06      0.11        49
    Standard       0.69      0.18      0.29       146
Super Deluxe       0.55      0.48      0.51        61

    accuracy                           0.56       978
   macro avg       0.54      0.41      0.42       978
weighted avg       0.56      0.56      0.53       978



In [62]:
from joblib import dump, load

In [63]:
dump(clf, 'model/RF003.joblib')

['model/RF003.joblib']

In [65]:
X_test.drop(['CUSTOMER_ID'], axis=1).head(1)

,AGE,OCCUPATION,GENDER,FAMILY_SIZE,DURATION_OF_STAY
0,28.0,2,1,3,15.0


In [66]:
clf.predict(X_test.drop(['CUSTOMER_ID'], axis=1).head(1))

array(['Basic'], dtype=object)

In [73]:
clf.predict([[28, 2, 1, 3, 15]])

/Users/bharatwaajshankar/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array(['Basic'], dtype=object)

In [99]:
from joblib import dump, load
from google.cloud import storage
import pprint
from tempfile import TemporaryFile

PROJECT_ID = 'serverless-bnb'
ML_MODEL_BUCKET = 'serverless-group20-ml-models'
MODEL_NAME = "RF003.joblib"

# Initialize the storage client
storage_client = storage.Client(project=PROJECT_ID)

def get_recommendations(data):
    
    """
        @author Bharatwaaj Shankaranarayanan
        @function get_recommendations
        @description This function saves the reviews data in the reviews data model
        @params data
        @returns response (JSON)
    """
    
    print("request_data", data)
    
    # Get recommendations
    try:
        bucket = storage_client.get_bucket(ML_MODEL_BUCKET)
        blob = bucket.get_blob(MODEL_NAME)
#         model_file = BytesIO()
#         blob.download_to_filename(model_file)
#         clf = load(model_file) 
        
        with TemporaryFile() as temp_file:
            #download blob into temp file
            blob.download_to_file(temp_file)
            temp_file.seek(0)
            #load into joblib
            clf=load(temp_file)
        
        result = clf.predict([[float(data["AGE"]), float(data["OCCUPATION"]), float(data["GENDER"]), float(data["FAMILY_SIZE"]), float(data["DURATION_OF_STAY"])]])
        
        return {
            "recommendation": result[0],
            "success": True
        }
        
    except Exception as e:
        pprint(e)

In [100]:
get_recommendations(
    {"AGE": "25", "OCCUPATION": 1, "GENDER": 0, "FAMILY_SIZE": "5", "DURATION_OF_STAY": "10"}
)

request_data {'AGE': '25', 'OCCUPATION': 1, 'GENDER': 0, 'FAMILY_SIZE': '5', 'DURATION_OF_STAY': '10'}


/Users/bharatwaajshankar/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


{'recommendation': 'Basic', 'success': True}

In [101]:
from joblib import dump, load
from google.cloud import storage
import pprint
from tempfile import TemporaryFile

PROJECT_ID = 'serverless-bnb'
ML_MODEL_BUCKET = 'serverless-group20-ml-models'
MODEL_NAME = "RF003.joblib"

In [104]:
bucket = storage_client.get_bucket(ML_MODEL_BUCKET)
blob = bucket.blob(MODEL_NAME)
with blob.open(mode="rb") as file:
    model = load(file)

In [ ]:
model.predict(X_test.drop()

In [105]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [106]:
import sys

In [108]:
!{sys.executable} -m pip install gcsfs

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.9/551.9 KB 8.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 KB 4.7 MB/s eta 0:00:00
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [117]:
bucket = storage_client.get_bucket(ML_MODEL_BUCKET)
blob = bucket.blob(MODEL_NAME)
with blob.open(mode="rb") as file:
    model = joblib.load(file)

In [118]:
model

RandomForestClassifier(max_depth=8, random_state=0)

In [112]:
import gcsfs, joblib

In [113]:
def load_joblib(bucket_name, file_name):
    fs = gcsfs.GCSFileSystem()
    with fs.open(f'{bucket_name}/{file_name}') as f:
        return joblib.load(f)

In [114]:
clf = load_joblib(ML_MODEL_BUCKET, MODEL_NAME)

In [ ]:
clf.predict()